In [ ]:
from matplotlib import pyplot as plt
import math,os,json,sys

import pandas as pd
import scipy
from sklearn.preprocessing import OneHotEncoder 
import numpy as np
from numpy.random import random, randn
import PIL
from PIL import Image

import keras
from keras import backend as K
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential  
from keras.layers.core import Flatten,Dropout,Dense
from keras.optimizers import Adam, SGD
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.callbacks import Callback
from clr_callback import *

from glob import glob
import bcolz
%matplotlib inline

In [ ]:
path="/home/kaggle/statefarm"

In [ ]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname,target_size=target_size,batch_size=batch_size,class_mode=class_mode,shuffle=shuffle)

def onehot(x):
    return to_categorical(x.classes,num_classes=10)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]    

In [ ]:
##Make validation sets 

for item in os.listdir(path+"/train"):
    path1=path+"/train/"+item
    %cd $path1
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(200): os.rename(shuf[i],path+'/valid/' + item +'/'+ shuf[i])

In [ ]:
batch_size=64
# batches shuffle must be set to False when pre-computing features
batches=get_batches(path +'/train',batch_size=batch_size,shuffle=False)
valid_batches=get_batches(path +'/valid',batch_size=batch_size*2,shuffle=False)
test_batches=get_batches(path +'/test',batch_size=32,shuffle=False)

In [ ]:
# model = Sequential([
#         BatchNormalization(axis=1, input_shape=(224,224,3)),
#         Convolution2D(32,3,3, activation='relu'),
#         BatchNormalization(axis=1),
#         MaxPooling2D((3,3)),
#         Convolution2D(64,3,3, activation='relu'),
#         BatchNormalization(axis=1),
#         MaxPooling2D((3,3)),
#         Convolution2D(128,3,3, activation='relu'),
#         BatchNormalization(axis=1),
#         MaxPooling2D((3,3)),
#         Flatten(),
#         Dense(200, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.5),
#         Dense(200, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.5),
#         Dense(10, activation='softmax')
#     ])

In [ ]:
# model.compile(Adam(lr=1e-5),loss="categorical_crossentropy",metrics=["accuracy"])
# model.fit_generator(batches,batches.samples/batch_size,epochs=2,validation_data=valid_batches,
#                     validation_steps=valid_batches.samples/valid_batches.batch_size)

In [ ]:
# model=VGG16(weights='imagenet',include_top=True)

In [ ]:
# gen_t=image.ImageDataGenerator(rotation_range=15,height_shift_range=0.05,shear_range=0.1,channel_shift_range=20,width_shift_range=0.1)
# batches=get_batches(path +'/train',batch_size=batch_size)

# model.layers.pop()
# for layer in model.layers: layer.trainable=False

In [ ]:
# last = model.layers[-1].output
# y = Dropout(0.3)(last)
# x = Dense(10, activation="softmax")(y)
# model = Model(model.input, x)

In [ ]:
# layers = model.layers
# # Get the index of the first dense layer...
# first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# # ...and set this and all subsequent layers to trainable
# for layer in layers[first_dense_idx:]: layer.trainable=True

In [ ]:
# model.compile(Adam(lr=1e-5),loss="categorical_crossentropy",metrics=["accuracy"])
# model.fit_generator(batches,batches.samples/batch_size,epochs=5,validation_data=valid_batches,
#                     validation_steps=valid_batches.samples/valid_batches.batch_size)

In [ ]:
# model.save_weights(path+'/models/da_conv8_1.h5')

In [ ]:
train_data_labels=onehot(batches)
val_data_labels=onehot(valid_batches)

In [ ]:
model=VGG16(weights='imagenet',include_top=True)
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [ ]:
conv_model = Sequential(conv_layers)

Pre-compute feature

In [ ]:
conv_train_feat=conv_model.predict_generator(batches, np.ceil(batches.samples/batches.batch_size))
conv_val_feat = conv_model.predict_generator(valid_batches, np.ceil(valid_batches.samples/valid_batches.batch_size))
conv_test_feat = conv_model.predict_generator(test_batches, np.ceil(test_batches.samples/test_batches.batch_size))

In [ ]:
save_array(path+'/results/conv_val_feat.dat', conv_val_feat)
save_array(path+'/results/conv_test_feat.dat', conv_test_feat)
save_array(path+'/results/conv_train_feat.dat', conv_train_feat)

In [ ]:
conv_feat = load_array(path+'/results/conv_train_feat.dat')
conv_val_feat = load_array(path+'/results/conv_val_feat.dat')
conv_test_feat=load_array(path+'/results/conv_test_feat.dat')

In [ ]:
#Data Augmentation
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
aug_batches = get_batches(path+'/train', gen_t, batch_size=batch_size, shuffle=False)

In [ ]:
aug_conv_feat = conv_model.predict_generator(aug_batches, 5*(np.ceil(aug_batches.samples/aug_batches.batch_size)))

In [ ]:
save_array(path+'/results/da_conv_feat2.dat', aug_conv_feat)

In [ ]:
aug_conv_feat = load_array(path+'/results/da_conv_feat2.dat')

In [ ]:
#concatenate them with our training data
aug_conv_feat = np.concatenate([aug_conv_feat, conv_feat])

In [ ]:
#concatenate them with our training labels
aug_trn_labels = np.concatenate([train_data_labels]*6)

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.4

## CYCLIC LEARNING RATE

In [ ]:
#Perform LR test to find the optimal learning rate range
clr = CyclicLR(base_lr=0.0001, max_lr=1, step_size=np.ceil(batches.samples/(batches.batch_size)), mode='triangular')

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(SGD(momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, train_data_labels, batch_size=batch_size, epochs=1, 
             validation_data=(conv_val_feat, val_data_labels),callbacks=[clr])

In [ ]:
history = clr.history

In [ ]:
plt.plot(history['lr'],history['acc'])
# plt.plot(acc)  
plt.title('Accuracy plot against accuracy')
plt.xscale('log')
plt.xticks
plt.ylabel('acc')
plt.xlabel('learning_rate')
plt.legend(['lr'])
plt.show()

By looking at the graph, the model starts converging when lr=0.02, so it is reasonable to set base lr = 0.01. Furthermore, above a learning rate of 0.05 the accuracy rise gets rough and eventually
begins to drop so it is reasonable to set max lr = 0.006.

In [ ]:
plt.plot(history['lr'],history['loss'])
# plt.plot(acc)  
plt.title('Learning rate plot against loss')
plt.xscale('log')
plt.ylabel('loss')
plt.xlabel('learning_rate')
plt.legend(['lr'])
plt.show()

In [ ]:
# clr = CyclicLR(base_lr=0.02, max_lr=0.05, step_size=np.ceil(aug_conv_feat.shape[0]/(batch_size)*2), mode='triangular') #for Vgg16
clr = CyclicLR(base_lr=0.005, max_lr=0.02, step_size=np.ceil(aug_conv_feat.shape[0]/(batch_size)*2), mode='triangular')  #For Vgg19

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(SGD(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(aug_conv_feat, aug_trn_labels, batch_size=batch_size, epochs=5, 
             validation_data=(conv_val_feat, val_data_labels),callbacks=[clr])

In [ ]:
bn_model.save_weights(path+'/models/conv8.h5')

In [ ]:
bn_model.load_weights(path+'/models/conv8.h5')

## PSEUDO-LABELLING

In [ ]:
result=bn_model.predict(conv_test_feat,batch_size=32)

In [ ]:
i_trn = 0
i_test = 0

# iterate through 800 mini-batch
num_iter = int(np.ceil(aug_conv_feat.shape[0]/(batch_size))*5)

size_trn=64
size_test=32

In [ ]:
num_iter

In [ ]:
num_batch_per_epoch_trn = int(aug_conv_feat.shape[0]/size_trn)
num_batch_per_epoch_test = int(conv_test_feat.shape[0]/size_test)

In [ ]:
# Create an shuffled index array
index_trn = np.random.permutation(num_batch_per_epoch_trn)
index_test = np.random.permutation(num_batch_per_epoch_test)

In [ ]:
for i in range(num_iter):
    # Get an index number from the shuffled index array for the current loop i
    i_trn = index_trn[i%num_batch_per_epoch_trn]
    i_test = index_test[i%num_batch_per_epoch_test] 

    comb_features = np.concatenate((aug_conv_feat[(size_trn*i_trn):size_trn*(i_trn+1)],
                                   conv_test_feat[(size_test*i_test):size_test*(i_test+1)]),axis=0)
    
    comb_labels = np.concatenate((aug_trn_labels[(size_trn*i_trn):size_trn*(i_trn+1)],
                                 result[(size_test*i_test):size_test*(i_test+1)]), axis=0)
    
    loss=bn_model.train_on_batch(comb_features, comb_labels)
    print(loss)
    # Shuffle index array after model had trained on the last mini-batch.
    if (i+1)%num_batch_per_epoch_trn == 0:
        index_trn = np.random.permutation(num_batch_per_epoch_trn)
    if (i+1)%num_batch_per_epoch_test == 0:
        index_test = np.random.permutation(num_batch_per_epoch_test)

In [ ]:
bn_model.save_weights(path+'/models/conv9.h5')

In [ ]:
bn_model.load_weights(path+'/models/conv9.h5')

In [ ]:
bn_model.fit(aug_conv_feat, aug_trn_labels, batch_size=batch_size, epochs=1, 
             validation_data=(conv_val_feat, val_data_labels),callbacks=[clr])

In [ ]:
result=bn_model.predict(conv_test_feat,batch_size=32)

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
subm = do_clip(result,0.99)
subm_name = path+'/results/result.csv'


In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
# submission = pd.DataFrame(result, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_batches.filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False)